In [15]:

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression

In [16]:
df = pd.read_csv('train.csv')

In [17]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [18]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [19]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [20]:
X = df.drop(columns=['Survived'])
y = df['Survived']

In [21]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2)

In [22]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
30,1,male,40.0,0,0,27.7208,C
10,3,female,4.0,1,1,16.7000,S
873,3,male,47.0,0,0,9.0000,S
182,3,male,9.0,4,2,31.3875,S
876,3,male,20.0,0,0,9.8458,S


In [23]:
numerical_features = ['Age', 'Fare']
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_features = ['Embarked', 'Sex']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ohe',OneHotEncoder(handle_unknown='ignore'))
])

In [24]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

In [25]:
clf=Pipeline(steps=[
    ('preprocessor',preprocessor),
    ('classifier',LogisticRegression())
])

In [26]:
from sklearn import set_config

set_config(display='diagram')
clf


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'Fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Embarked', 'Sex'])])),
                ('classifier', LogisticRegression())])

In [27]:
param_grid = {
    'preprocessor__num__imputer__strategy': ['mean', 'median'],
    'preprocessor__cat__imputer__strategy': ['most_frequent', 'constant'],
    'classifier__C': [0.1, 1.0, 10, 100]
}

grid_search = GridSearchCV(clf, param_grid, cv=10)

In [29]:
grid_search.fit(X_train, y_train)

print(f"Best params:")
print(grid_search.best_params_)

Best params:
{'classifier__C': 1.0, 'preprocessor__cat__imputer__strategy': 'most_frequent', 'preprocessor__num__imputer__strategy': 'mean'}


In [30]:
print(f"Internal CV score: {grid_search.best_score_:.3f}")

Internal CV score: 0.788


In [31]:
import pandas as pd

cv_results = pd.DataFrame(grid_search.cv_results_)

In [32]:
cv_results = cv_results.sort_values("mean_test_score", ascending=False)

In [33]:
cv_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__C,param_preprocessor__cat__imputer__strategy,param_preprocessor__num__imputer__strategy,params,split0_test_score,split1_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
4,0.016401,0.002107,0.007201,0.001078,1.0,most_frequent,mean,"{'classifier__C': 1.0, 'preprocessor__cat__imp...",0.763889,0.861111,...,0.802817,0.788732,0.760563,0.84507,0.732394,0.760563,0.788732,0.787852,0.037656,1
5,0.018000,0.002720,0.007700,0.003227,1.0,most_frequent,median,"{'classifier__C': 1.0, 'preprocessor__cat__imp...",0.763889,0.861111,...,0.802817,0.788732,0.760563,0.84507,0.732394,0.760563,0.788732,0.787852,0.037656,1
6,0.018600,0.003774,0.006302,0.002370,1.0,constant,mean,"{'classifier__C': 1.0, 'preprocessor__cat__imp...",0.763889,0.861111,...,0.802817,0.788732,0.760563,0.84507,0.732394,0.760563,0.788732,0.787852,0.037656,1
7,0.019199,0.002358,0.006802,0.001602,1.0,constant,median,"{'classifier__C': 1.0, 'preprocessor__cat__imp...",0.763889,0.861111,...,0.802817,0.788732,0.760563,0.84507,0.732394,0.760563,0.788732,0.787852,0.037656,1
8,0.016201,0.002041,0.006602,0.002059,10.0,most_frequent,mean,"{'classifier__C': 10, 'preprocessor__cat__impu...",0.763889,0.861111,...,0.802817,0.788732,0.760563,0.84507,0.732394,0.760563,0.788732,0.787852,0.037656,1
9,0.019400,0.002245,0.007801,0.001662,10.0,most_frequent,median,"{'classifier__C': 10, 'preprocessor__cat__impu...",0.763889,0.861111,...,0.802817,0.788732,0.760563,0.84507,0.732394,0.760563,0.788732,0.787852,0.037656,1
10,0.017702,0.002369,0.006798,0.001661,10.0,constant,mean,"{'classifier__C': 10, 'preprocessor__cat__impu...",0.763889,0.861111,...,0.802817,0.788732,0.760563,0.84507,0.732394,0.760563,0.788732,0.787852,0.037656,1
11,0.019098,0.003643,0.006302,0.001903,10.0,constant,median,"{'classifier__C': 10, 'preprocessor__cat__impu...",0.763889,0.861111,...,0.802817,0.788732,0.760563,0.84507,0.732394,0.760563,0.788732,0.787852,0.037656,1
12,0.030400,0.038394,0.008101,0.006458,100.0,most_frequent,mean,"{'classifier__C': 100, 'preprocessor__cat__imp...",0.763889,0.861111,...,0.802817,0.788732,0.760563,0.84507,0.732394,0.760563,0.788732,0.787852,0.037656,1
13,0.019502,0.004610,0.006998,0.001897,100.0,most_frequent,median,"{'classifier__C': 100, 'preprocessor__cat__imp...",0.763889,0.861111,...,0.802817,0.788732,0.760563,0.84507,0.732394,0.760563,0.788732,0.787852,0.037656,1


In [34]:
cv_results[['param_classifier__C','param_preprocessor__cat__imputer__strategy','param_preprocessor__num__imputer__strategy','mean_test_score']]

,param_classifier__C,param_preprocessor__cat__imputer__strategy,param_preprocessor__num__imputer__strategy,mean_test_score
4,1.0,most_frequent,mean,0.787852
5,1.0,most_frequent,median,0.787852
6,1.0,constant,mean,0.787852
7,1.0,constant,median,0.787852
8,10.0,most_frequent,mean,0.787852
9,10.0,most_frequent,median,0.787852
10,10.0,constant,mean,0.787852
11,10.0,constant,median,0.787852
12,100.0,most_frequent,mean,0.787852
13,100.0,most_frequent,median,0.787852
